In [ ]:
!pip install git+https://github.com/nadavbra/protein_bert.git

  Cloning https://github.com/nadavbra/protein_bert.git to /tmp/pip-req-build-q9910nsm
  Running command git clone --filter=blob:none --quiet https://github.com/nadavbra/protein_bert.git /tmp/pip-req-build-q9910nsm
  Resolved https://github.com/nadavbra/protein_bert.git to commit 168a4db5aac281ff14165d00e50f862d780a8966
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 11.3 MB/s eta 0:00:00
  Created wheel for protein-bert: filename=protein_bert-1.0.1-py3-none-any.whl size=39374 sha256=ba3d09e25eb9357d7b930ac3befcdecf61a344d1b29e2c4887c2713eade9efaa
  Stored in directory: /tmp/pip-ephem-wheel-cache-lca2xznn/wheels/5f/5c/b8/286c3fbf08540f4457e4239bf02c9c6dd129a5b0e80e34d80e
Successfully built protein-bert


In [ ]:
import proteinbert
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
from sklearn.model_selection import train_test_split

import tensorflow
from tensorflow import keras
import json

In [ ]:
print("Num GPUs Available:", len(tensorflow.config.experimental.list_physical_devices('GPU')))

Num GPUs Available: 1


In [ ]:
json_path = "/content/drive/MyDrive/protein_props.json"
with open(json_path) as f:
  data = json.load(f)
len(data)

20434

In [ ]:
okk = pd.read_json(json_path)
properties = okk.transpose()

In [ ]:
druggable_data = "/content/drive/MyDrive/druggable_proteins.txt"
with open(druggable_data) as f:
  druggable_proteins = [line.strip() for line in f]
print(len(druggable_proteins))

approved_drugs = "/content/drive/MyDrive/approved_druggable_proteins.txt"
with open(approved_drugs) as g:
  approved_proteins = [line.strip() for line in g]
print(len(approved_proteins))

3345
2652


In [ ]:
is_druggable = []
for i in properties.index:
  if i in druggable_proteins:
    is_druggable.append(1)
  else:
    is_druggable.append(0)

approved_drugs = []
for j in properties.index:
  if j in approved_proteins:
    approved_drugs.append(1)
  else:
    approved_drugs.append(0)

properties['is_druggable'] = is_druggable
properties['is_approved'] = approved_drugs
properties

,Sequence,Sequence Length,Molecular Weight,GRAVY,Amino Acid Count,Amino Acid Percent,Molar Extinction Coefficient,Isoelectric Point,Instability Index,Aromaticity,Secondary Structure,Flexibility,Charge at 7,is_druggable,is_approved
A0A087X1C5,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,515,57488.0269,-0.016117,"{'A': 42, 'C': 9, 'D': 23, 'E': 28, 'F': 31, '...","{'A': 0.081553398058252, 'C': 0.01747572815533...","[47440, 47940]",8.703506,45.155922,0.085437,"[0.31456310679611604, 0.262135922330097, 0.378...","[0.977440476190476, 0.9806071428571421, 0.9983...",5.889114,0,0
A0A0B4J2F0,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...,54,6313.3024,-0.235185,"{'A': 4, 'C': 0, 'D': 1, 'E': 6, 'F': 5, 'G': ...","{'A': 0.074074074074074, 'C': 0.0, 'D': 0.0185...","[2980, 2980]",8.03542,55.518519,0.12963,"[0.44444444444444403, 0.129629629629629, 0.388...","[0.982095238095238, 0.95625, 0.983214285714285...",0.510326,0,0
A0A0B4J2F2,MVIMSEFSADPAGQGQGQQKPLRVGFYDIERTLGKGNFAVVKLARH...,783,84929.1856,-0.197957,"{'A': 66, 'C': 19, 'D': 30, 'E': 43, 'F': 28, ...","{'A': 0.08429118773946301, 'C': 0.024265644955...","[56840, 57965]",6.813029,59.277803,0.063857,"[0.314176245210727, 0.31289910600255405, 0.328...","[1.00125, 0.970071428571428, 1.034523809523809...",-1.004586,0,0
A0A0C5B5G6,MRWQEMGYIFYPRKLR,16,2174.5917,-0.9375,"{'A': 0, 'C': 0, 'D': 0, 'E': 1, 'F': 1, 'G': ...","{'A': 0.0, 'C': 0.0, 'D': 0.0, 'E': 0.0625, 'F...","[8480, 8480]",10.266413,77.300625,0.25,"[0.3125, 0.125, 0.375]","[0.9704047619047621, 1.00345238095238, 0.95860...",2.500138,0,0
A0A0K2S4Q6,MTQRAGAAMLPSALLLLCVPGCLTVSGPSTVMGAVGESLSVQCRYE...,201,21805.9293,0.10796,"{'A': 11, 'C': 8, 'D': 7, 'E': 10, 'F': 6, 'G'...","{'A': 0.054726368159203, 'C': 0.03980099502487...","[33460, 33960]",5.366988,41.796517,0.074627,"[0.283582089552238, 0.308457711442786, 0.40298...","[1.008285714285714, 0.9868928571428571, 0.991,...",-3.378625,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9UI54,MESPKCLYSRITVNTAFGTKFSHISFIILFKVFLFPRITISKKTKL...,55,6386.613,0.303636,"{'A': 1, 'C': 1, 'D': 0, 'E': 1, 'F': 6, 'G': ...","{'A': 0.018181818181818, 'C': 0.01818181818181...","[2980, 2980]",10.302902,35.789091,0.145455,"[0.29090909090909, 0.218181818181818, 0.527272...","[0.9746904761904761, 1.00325, 0.98202380952380...",7.571427,0,0
Q9UI72,MGMALELYWLCGFRSYWPLGTNAENEGNRKENRRQMQSRNERGCNV...,69,8249.2539,-1.015942,"{'A': 4, 'C': 3, 'D': 2, 'E': 6, 'F': 2, 'G': ...","{'A': 0.057971014492753006, 'C': 0.04347826086...","[15470, 15595]",9.348383,46.171014,0.101449,"[0.30434782608695604, 0.24637681159420202, 0.2...","[1.006333333333333, 0.9435238095238091, 0.9573...",3.659009,0,0
Q9Y3F1,MSLLWTPQILTISFVSYILSLFPSPFPSCYTSCWFETSITTEKELN...,56,6534.5215,0.444643,"{'A': 2, 'C': 2, 'D': 0, 'E': 4, 'F': 6, 'G': ...","{'A': 0.035714285714285005, 'C': 0.03571428571...","[15470, 15595]",4.493088,69.675,0.196429,"[0.30357142857142805, 0.232142857142857, 0.535...","[0.988154761904761, 0.9863214285714281, 0.9781...",-2.5132,0,0
Q9Y6C7,MAHHSLNTFYIWHNNVLHTHLVFFLPHLLNQPFSRGSFLIWLLLCW...,94,11023.7471,-0.029787,"{'A': 4, 'C': 2, 'D': 1, 'E': 1, 'F': 5, 'G': ...","{'A': 0.042553191489361, 'C': 0.02127659574468...","[24980, 25105]",10.192468,62.375638,0.117021,"[0.319148936170212, 0.24468085106382903, 0.436...","[0.9616666666666661, 1.001988095238095, 0.9683...",6.176832,0,0


In [ ]:
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(properties['Sequence'], properties['is_druggable'] , test_size = 0.2, random_state = 123)
X_train,X_val,y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 124)

In [ ]:
np.sort(properties['Sequence Length'].values)[-50:]

array([4624, 4646, 4655, 4678, 4684, 4700, 4753, 4834, 4857, 4861, 4870,
       4911, 4967, 4981, 5005, 5038, 5058, 5079, 5088, 5121, 5142, 5150,
       5183, 5202, 5207, 5207, 5289, 5405, 5412, 5478, 5537, 5596, 5635,
       5654, 5762, 5795, 5890, 6306, 6669, 6885, 6907, 7081, 7388, 7570,
       7968, 8384, 8797, 13477, 14507, 34350], dtype=object)

In [ ]:
pretrained_model, input_encoder = load_pretrained_model()

 Local model dump file /root/proteinbert_models/default.pkl doesn't exist. Will download https://media.githubusercontent.com/media/nadavbra/proteinbert_data_files/master/epoch_92400_sample_23500000.pkl into /root/proteinbert_models. Please approve or reject this (to exit and potentially call the function again with different parameters).
Do you approve downloading the file into the specified directory? Please specify "Yes" or "No":Yes
Downloaded file: /root/proteinbert_models/epoch_92400_sample_23500000.pkl
Created: /root/proteinbert_models/default.pkl


In [ ]:
model_generator = FinetuningModelGenerator(pretrained_model, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

In [ ]:
#to ease the fine_tuning process

training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-05, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
]

In [ ]:
finetune(model_generator, input_encoder, OUTPUT_SPEC, X_train, y_train, X_val, y_val, \
        seq_len = 3000, batch_size = 32, max_epochs_per_stage = 50, lr = 1e-04, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 6000, final_lr = 1e-05, callbacks = training_callbacks)



[2024_03_28-03:58:08] Training set: Filtered out 123 of 14712 (0.8%) records of lengths exceeding 2998.
[2024_03_28-03:58:11] Validation set: Filtered out 13 of 1635 (0.8%) records of lengths exceeding 2998.
[2024_03_28-03:58:11] Training with frozen pretrained layers...
Epoch 1/50
456/456 [==============================] - ETA: 0s - loss: 0.3751

456/456 [==============================] - 180s 368ms/step - loss: 0.3751 - val_loss: 0.3517 - lr: 0.0100
Epoch 2/50
456/456 [==============================] - ETA: 0s - loss: 0.3712


Epoch 2: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
456/456 [==============================] - 158s 347ms/step - loss: 0.3712 - val_loss: 0.3577 - lr: 0.0100
Epoch 3/50
456/456 [==============================] - ETA: 0s - loss: 0.3324


Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.
456/456 [==============================] - 162s 356ms/step - loss: 0.3324 - val_loss: 0.3519 - lr: 0.0025
[2024_03_28-04:06:34] Training the entire fine-tuned model...
[2024_03_28-04:07:13] Incompatible number of optimizer weights - will not initialize them.
Epoch 1/50
456/456 [==============================] - ETA: 0s - loss: 0.3337

456/456 [==============================] - 424s 907ms/step - loss: 0.3337 - val_loss: 0.3306 - lr: 1.0000e-04
Epoch 2/50
456/456 [==============================] - ETA: 0s - loss: 0.3108

456/456 [==============================] - 411s 901ms/step - loss: 0.3108 - val_loss: 0.3184 - lr: 1.0000e-04
Epoch 3/50
456/456 [==============================] - ETA: 0s - loss: 0.2872


Epoch 3: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
456/456 [==============================] - 411s 900ms/step - loss: 0.2872 - val_loss: 0.3470 - lr: 1.0000e-04
Epoch 4/50
456/456 [==============================] - ETA: 0s - loss: 0.2422


Epoch 4: ReduceLROnPlateau reducing learning rate to 1e-05.
456/456 [==============================] - 416s 912ms/step - loss: 0.2422 - val_loss: 0.3306 - lr: 2.5000e-05
[2024_03_28-04:34:54] Training on final epochs of sequence length 6000...
[2024_03_28-04:34:54] Training set: Filtered out 11 of 14712 (0.1%) records of lengths exceeding 5998.
[2024_03_28-04:34:59] Validation set: Filtered out 0 of 1635 (0.0%) records of lengths exceeding 5998.
919/919 [==============================] - ETA: 0s - loss: 0.2864

919/919 [==============================] - 870s 937ms/step - loss: 0.2864 - val_loss: 0.3224 - lr: 1.0000e-05


In [ ]:
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, X_test, y_test, \
        start_seq_len = 512, start_batch_size = 32, increase_factor = 100)

1527/1527 [==============================] - 263s 171ms/step


In [ ]:
results

,# records,AUC
Model seq len,,
512,2560,0.812014
51200,1527,0.736963
All,4087,0.772607


In [ ]:
confusion_matrix.values

array([[3178,  191],
       [ 516,  202]])

In [ ]:
accuracy = (3178 + 202)/(3178 + 202 + 519 + 191)
accuracy

0.8264058679706602